In [3]:
import sys
sys.path.append('/workspaces/bha2/src')

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import nibabel as nib
from nilearn import plotting
from connectivity_functions import *
from tree_functions import *
from ni_functions import *
from io_functions import load_data, load_receptor_data
import os
import json
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20, 10)
import seaborn as sns

/opt/conda/lib/python3.9/site-packages/nilearn/input_data/__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [6]:
project_path = "/workspaces/bha2"
conn_size_small = 183
conn_size_big = 2165

## First we compute the xmod without building the tree using a parcellation with small number of rois

In [7]:
if os.path.exists(os.path.join(project_path, "tmp", "n" + str(conn_size_small) + "_fcm.npy")):
    fcm = np.load(os.path.join(project_path, "tmp", "n" + str(conn_size_small) + "_fcm.npy"))
    scm = np.load(os.path.join(project_path, "tmp", "n" + str(conn_size_small) + "_scm.npy"))
    print("fcm and scm loaded from tmp folder")
else:
    sc_group = load_data(os.path.join(project_path, "data", "raw", "n" + str(conn_size_small), "sc"))
    fc_group = load_data(os.path.join(project_path, "data", "raw", "n" + str(conn_size_small), "fc"))
    fcm, scm = connectome_average(fc_group, sc_group)
    np.save(os.path.join(project_path, "tmp", "n" + str(conn_size_small) + "_fcm.npy"), fcm)
    np.save(os.path.join(project_path, "tmp", "n" + str(conn_size_small) + "_scm.npy"), scm)

fcm and scm loaded from tmp folder


In [8]:
zero_rows_sc = np.where(~scm.any(axis=1))[0]
fcm_nonzero, scm_nonzero = remove_rois_from_connectomes(zero_rows_sc, fcm, scm)
density = np.where(scm_nonzero.flatten() > 0, 1, 0).sum(dtype=float) / (len(scm_nonzero.flatten()))
fcm_thr = density_threshold(fcm_nonzero, density)
zero_rows_fc = np.where(~fcm_thr.any(axis=1))[0]
fcm_nonzero, scm_nonzero = remove_rois_from_connectomes(zero_rows_fc, fcm_thr, scm_nonzero)

print("connectome density = " + str(density))

connectome density = 0.59249305742184


In [ ]:
sim = similarity_mean_level(fcm_nonzero, scm_nonzero, level)
mod_sc_lvl = modularity(scm_nonzero, T)
mod_fc_lvl = modularity(fcm_nonzero, T)
xmod = pow((sim * mod_sc_lvl * mod_fc_lvl), (1 / 3))